In [1]:
import numpy as np
import pandas as pd
import gc
from sklearn.preprocessing import Imputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif

import matplotlib.pyplot as plt
import seaborn as sns
from plotly import offline
import plotly.plotly as py
import plotly.graph_objs as go
color = sns.color_palette()

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Load data

In [2]:
install = pd.read_csv('../input/installments_payments.csv')

In [5]:
print('Installment payments data shape: ', install.shape)
install.head()

Installment payments data shape:  (13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [8]:
install.dtypes

SK_ID_PREV                  int64
SK_ID_CURR                  int64
NUM_INSTALMENT_VERSION    float64
NUM_INSTALMENT_NUMBER       int64
DAYS_INSTALMENT           float64
DAYS_ENTRY_PAYMENT        float64
AMT_INSTALMENT            float64
AMT_PAYMENT               float64
dtype: object

In [9]:
# no anomaly in the days
DAYS_col = [col for col in install if col.startswith('DAYS')]
install[DAYS_col].max()

DAYS_INSTALMENT      -1.0
DAYS_ENTRY_PAYMENT   -1.0
dtype: float64

In [11]:
na_totals = install.isnull().sum().sort_values(ascending=False)
print ("Number of NAs in the columns:")
na_totals[na_totals>0]

Number of NAs in the columns:


AMT_PAYMENT           2905
DAYS_ENTRY_PAYMENT    2905
dtype: int64

In [3]:
# replace missing values with 0 as they never pay
install['AMT_PAYMENT'].fillna(0, inplace=True)
install['DAYS_ENTRY_PAYMENT'].fillna(0, inplace=True)

In [91]:
install.groupby(['SK_ID_CURR']).size().describe()

count    339587.000000
mean         40.064552
std          41.053343
min           1.000000
25%          12.000000
50%          25.000000
75%          51.000000
max         372.000000
dtype: float64

In [15]:
install[install['SK_ID_CURR']==100001].sort_values('DAYS_INSTALMENT')

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
1761012,1851984,100001,1.0,2,-2916.0,-2916.0,3982.050,3982.050
3774071,1851984,100001,1.0,3,-2886.0,-2875.0,3982.050,3982.050
3435373,1851984,100001,1.0,4,-2856.0,-2856.0,3980.925,3980.925
1478621,1369693,100001,1.0,1,-1709.0,-1715.0,3951.000,3951.000
2568722,1369693,100001,1.0,2,-1679.0,-1715.0,3951.000,3951.000
3458712,1369693,100001,1.0,3,-1649.0,-1660.0,3951.000,3951.000
2624024,1369693,100001,2.0,4,-1619.0,-1628.0,17397.900,17397.900


client 100001 was late on second installment payment, and then also late on last four payments. First few payments are crucial because if a client was late in the beginning, collection would send reminder for later payments. So we need a way to capture the promptness of clients' first few payments.

In [92]:
install['SK_ID_CURR'].unique()

array([161674, 151639, 193053, ..., 413433, 434445, 405063])

In [5]:
ins = install.sort_values('DAYS_INSTALMENT').copy()
# Percentage and difference paid in each installment (amount paid and installment value)
# ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
# Days past due and days before due (no negative values)
ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)

# Features: Perform aggregations
def head_sum_3(x):
    return x.head(3).sum()

def head_sum_5(x):
    return x.head(5).sum()

aggregations = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DPD': ['max', 'mean', 'sum'],
    'DBD': ['max', 'mean', 'sum', head_sum_3, head_sum_5],
    #'PAYMENT_PERC': ['max', 'mean', 'sum'],
    'PAYMENT_DIFF': ['max', 'mean', 'sum', head_sum_3,head_sum_5],
    'AMT_INSTALMENT': ['max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
    'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
}

ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
# Count installments accounts
ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()

In [6]:
application_train = pd.read_csv("../input/application_train.csv") 
application_test = pd.read_csv("../input/application_test.csv")

df_index = application_train.append(application_test).reset_index(drop=True)[['SK_ID_CURR','TARGET']]
df_joined = df_index.join(ins_agg, how='left', on='SK_ID_CURR')
# clear the memory
del application_train
del application_test
del df_index
gc.collect()

52

In [75]:
na_totals = df_joined.isnull().sum().sort_values(ascending=False)
print ("Number of NAs in the columns:")
na_totals[na_totals>0]

Number of NAs in the columns:


TARGET                                   48744
INSTAL_COUNT                             16668
INSTAL_PAYMENT_DIFF_MEAN                 16668
INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE    16668
INSTAL_DPD_MAX                           16668
INSTAL_DPD_MEAN                          16668
INSTAL_DPD_SUM                           16668
INSTAL_DBD_MAX                           16668
INSTAL_DBD_MEAN                          16668
INSTAL_DBD_SUM                           16668
INSTAL_PAYMENT_DIFF_MAX                  16668
INSTAL_PAYMENT_DIFF_SUM                  16668
INSTAL_DAYS_ENTRY_PAYMENT_SUM            16668
INSTAL_AMT_INSTALMENT_MAX                16668
INSTAL_AMT_INSTALMENT_MEAN               16668
INSTAL_AMT_INSTALMENT_SUM                16668
INSTAL_AMT_PAYMENT_MIN                   16668
INSTAL_AMT_PAYMENT_MAX                   16668
INSTAL_AMT_PAYMENT_MEAN                  16668
INSTAL_AMT_PAYMENT_SUM                   16668
INSTAL_DAYS_ENTRY_PAYMENT_MAX            16668
INSTAL_DAYS_E

In [7]:
def fill_with_median(df, missing_cols):
    # add a new column signifying missing install records
    df['INSTALL_MISSING'] = pd.isnull(missing_cols[0])
    # replace missing values with median
    for col in missing_cols:
        col_m = df[col].median()
        df[col].fillna(col_m, inplace=True)

na_totals = df_joined.isnull().sum().sort_values(ascending=False)
missing_cols = list(na_totals[na_totals==16668].index)
fill_with_median(df_joined, missing_cols)

In [8]:
train_labels = df_joined[df_joined['TARGET'].notnull()].TARGET.values
train_df = df_joined[df_joined['TARGET'].notnull()].drop(['TARGET','SK_ID_CURR'],axis=1)
test_df = df_joined[df_joined['TARGET'].isnull()].drop(['TARGET','SK_ID_CURR'],axis=1)

In [9]:
train_df.shape

(307511, 26)

In [10]:
VarianceThreshold(.01).fit_transform(train_df).shape  # INSTALL_MISSING would be removed

(307511, 25)

In [11]:
random_forest = RandomForestClassifier(n_estimators = 20, random_state = 50, verbose = 1, n_jobs = -1)

print('Cross validation ROC AUC score is: ',
      cross_val_score(random_forest, train_df, train_labels, scoring='roc_auc').mean())
# 0.576

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    5.0s finished
[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    5.5s finished
[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    5.5s finished


Cross validation ROC AUC score is:  0.5775853554874483


[Parallel(n_jobs=8)]: Done  20 out of  20 | elapsed:    0.1s finished


In [12]:
df_joined[['SK_ID_CURR'] + list(train_df.columns)].to_csv('../input/install_features.csv', index = False)